# Imports

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
import plotly.graph_objects as go

from scripts import Datax18, NARXx18



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data preprocessing

In [2]:
# make df with column names year, month, decimal date, SSN, M1, M2, M3, M4, M5 from file 5c_obs_sim_v1.dat
df = pd.read_csv('5c_obs_sim_v3.dat', sep='\s+', header=None, names=['decimal_date', 'SSN', 'M1', 'M2', 'M3', 'M4', 'M5'])

# normalize data to be between 0 and 1 using max value from SSN
df['SSN'] = df['SSN']
for i in range(1, 6):
    df[f'M{i}'] = df[f'M{i}']

df.head()

,decimal_date,SSN,M1,M2,M3,M4,M5
0,1964.290,18.6,6.238113,1.358705,8.952250,20.055768,15.689878
1,1964.373,16.0,6.552118,1.509507,9.384701,20.675201,14.810065
2,1964.456,15.0,6.888593,1.682225,9.867576,21.396530,14.318844
3,1964.540,15.2,7.252599,1.858949,10.374440,22.039826,14.351114
4,1964.624,15.1,7.667502,2.059977,10.954735,22.754826,14.813335


In [3]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['decimal_date'], y=df['SSN'], name='SSN'))

for i in range(1, 6):
    fig.add_trace(go.Scatter(x=df['decimal_date'], y=df[f'M{i}'], name=f'M{i}', visible=True))

# fig.update_yaxes(range=[-0.2, ])
fig.show()

In [4]:
model_series = df['M1'].values
observed_series = df['SSN'].values

In [46]:
# find first -1 in observed series
first_nan = np.where(df['SSN'] == -1)[0][0]

# find first decimal date >= 2024
first_2024 = np.where(df['decimal_date'] >= 2024)[0][0]

first_2024, first_nan

(717, 722)

717

In [32]:
observed_series = observed_series[:first_nan]
model_series = model_series[:first_nan]

In [5]:
M = np.max(observed_series)
observed_series = observed_series / M
model_series = model_series / M

In [7]:
prev_values = 4
horizon = 18
data = Datax18(model_series, observed_series, prev_values, horizon)

In [9]:
data.X.shape

torch.Size([833, 26])

In [10]:
train_data_X = data.X[:710 + 1 - (horizon - 1) - prev_values]
train_data_y = data.y[:710 + 1 - (horizon - 1) - prev_values]
train_data_X.shape, train_data_y.shape

(torch.Size([690, 26]), torch.Size([690]))

In [97]:
train_data_y * M

tensor([ 39.7000,  44.8000,  49.2000,  53.3000,  57.9000,  63.4000,  71.4000,
         80.3000,  89.5000,  95.8000,  99.4000, 103.0000, 106.2000, 111.6000,
        116.5000, 119.8000, 123.9000, 129.4000, 133.3000, 135.0000, 134.9000,
        134.5000, 137.6000, 142.5000, 145.3000, 145.7000, 148.2000, 151.8000,
        152.4000, 150.9000, 148.9000, 148.4000, 151.5000, 155.5000, 156.6000,
        156.0000, 155.7000, 155.1000, 152.9000, 150.7000, 150.4000, 150.1000,
        149.9000, 150.7000, 149.2000, 147.4000, 148.0000, 148.5000, 149.5000,
        150.1000, 150.3000, 150.3000, 149.8000, 149.1000, 147.0000, 143.0000,
        137.6000, 132.9000, 126.5000, 119.0000, 113.8000, 110.1000, 105.3000,
        100.4000,  96.5000,  94.5000,  92.7000,  91.5000,  93.2000,  93.8000,
         94.7000,  98.3000, 100.3000, 100.9000, 102.6000, 104.0000, 103.3000,
         99.9000,  96.6000,  92.9000,  88.2000,  85.9000,  83.3000,  78.2000,
         72.3000,  66.1000,  62.8000,  60.8000,  57.9000,  55.60

In [98]:
train_data_X * M

tensor([[ 6.2381,  6.5521,  6.8886,  ..., 16.0000, 15.0000, 15.2000],
        [ 6.5521,  6.8886,  7.2526,  ..., 15.0000, 15.2000, 15.1000],
        [ 6.8886,  7.2526,  7.6675,  ..., 15.2000, 15.1000, 14.6000],
        ...,
        [35.5940, 38.0349, 40.7088,  ..., 35.4000, 40.2000, 45.2000],
        [38.0349, 40.7088, 43.5622,  ..., 40.2000, 45.2000, 50.8000],
        [40.7088, 43.5622, 46.5515,  ..., 45.2000, 50.8000, 55.9000]])

In [16]:
data.y[710 + 1 - (horizon - 1) - prev_values + 10] * M

tensor(149.1000)

In [17]:
test_data_X = data.X[710 + 1 - (horizon - 1) - prev_values: 710 + 1 - (horizon - 1) - prev_values + 12]

In [6]:
path_to_models = 'real_model/'

predictions = []

for horizon in range (1, 19):
    model = NARXx18(2 * 4 + horizon, [24], 1, M)
    model.load_state_dict(torch.load(os.path.join(path_to_models, f'model_weights_horizon{horizon}.pth')))
    data = Datax18(model_series, observed_series, 4, horizon)
    test_data_X = data.X[710 + 1 - (horizon - 1) - 4: 710 + 1 - (horizon - 1) - 4 + 12]
    test_data_X = test_data_X.to(device)
    model = model.to(device)
    res = model(test_data_X).squeeze()
    res = res.cpu().detach().numpy()
    res = res * M
    predictions.append(res)
    
predictions = np.array(predictions)
    


In [13]:
# Год начала
start_year = 23
start_month = 6

# Создаем файлы CSV
for i in range(12):
    year = 24
    month = i + 1
    file_name = f"R_{year}_{str(month).zfill(2)}.txt"

    # Подготавливаем данные для текущего месяца
    # data = []
    # for j in range(18):
    #     current_year = start_year + (start_month - 1 + j) // 12
    #     current_month = (start_month + j - 1) % 12 + 1
    #     data.append(predictions.T[i][j])

    # # # Создаем txt файл
    # # with open(file_name, "w") as file:
    # #     # write prediction in each line
    # #     for j in range(18):
    # #         file.write(f"{data[j]}\n")
    
    with open(file_name, "w") as file:
        for j in range(18):
            file.write(f"{predictions.T[i][j]}\n")
    

In [11]:
data

[151.78685,
 149.32854,
 148.17278,
 145.21333,
 137.76703,
 139.91512,
 139.43532,
 133.67186,
 130.07246,
 133.1005,
 129.55045,
 138.49898,
 139.96434,
 140.28395,
 139.60385,
 137.16345,
 154.13542,
 137.6751]